# Installing packages

In [1]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install "torch==2.4.0" "xformers==0.0.27.post2" triton torchvision torchaudio "trl==0.14.0"
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"
!pip uninstall -y trl
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b

# Import Libs

In [2]:
import numpy as np
import pandas as pd

import torch
import torch.nn.functional as F
from unsloth import FastLanguageModel

import os
import ast
import string
import random
from tqdm.auto import tqdm
from IPython.display import clear_output

os.environ["WANDB_DISABLED"] = "true"
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


# Downloading model

In [3]:
model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"

# >>> Download LLM Model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 8192,
    dtype = None,
    load_in_4bit = True,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)
FastLanguageModel.for_inference(model)
print("Model is ready to inference!")

==((====))==  Unsloth 2025.3.19: Fast Mistral patching. Transformers: 4.50.2.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.4.0+cu121. CUDA: 6.0. CUDA Toolkit: 12.1. Triton: 3.0.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Model is ready to inference!


# Load & process dataset

In [4]:
df_train = pd.read_csv("/kaggle/input/fpt-ai-residency-batch-6-entry-test/b6_train_data.csv")
df_train.head()

,task_id,question,choices,answer
0,k10168,Question: What will be output of the following...,"['8 4 2', '8 4 2', '8 4 4', '8 4 3']",C
1,k10173,Question: What will be output of the following...,"['-4', '-5', '10', '11']",A
2,k10174,Question: Match the following.\n Group 1 ...,"['P-4. Q-1, R-2, S-3', 'P-3, Q-1, R-4, S-2', '...",B
3,k10175,Question: Match the following.\nP. Regular exp...,"['P-4. Q-1, R-2, S-3', 'P-3, Q-1, R-4, S-2', '...",B
4,k10176,Question: Which grammar rules violate the requ...,"['1 only', '1 and 3 only', '2 and 3 only', '3 ...",B


In [5]:
df_test = pd.read_csv("/kaggle/input/fpt-ai-residency-batch-6-entry-test/b6_test_data.csv")
df_test.head()

,task_id,question,choices
0,k09698,Question: Consider the following array of elem...,"['4', '5', '2', '3']"
1,k00203,Question: The SVG <polygon> element is used to...,"['False', 'True']"
2,k00137,Question: Only one of the following statements...,['The view SHOULD NOT be affected by the chang...
3,k10490,Question: Which operator is used to multiply n...,"['x', '%', '*', '#']"
4,rt03960,"Question: Given a code snippet below, which be...","['Compile Error', 'Runtime Error', 'Internal e..."


In [6]:
df_train.describe()

,task_id,question,choices,answer
count,3963,3963,3963,3949
unique,3963,3875,3451,11
top,k00701,Question: What is Conditional Rendering?\n['It...,"['False', 'True']",A
freq,1,7,22,988


In [7]:
df_train = df_train.dropna()
df_train = df_train.drop_duplicates()
df_train.describe()

,task_id,question,choices,answer
count,3949,3949,3949,3949
unique,3949,3863,3450,11
top,k00701,Question: What is Conditional Rendering?\n['It...,"['False', 'True']",A
freq,1,7,22,988


In [8]:
df_test.describe()

,task_id,question,choices
count,1253,1253,1253
unique,1253,1245,1163
top,k09340,Question: Assume that Source S and Destination...,"['(1)', '(2)', '(3)', '(4)']"
freq,1,2,7


In [68]:
solve_template = """You are a programming expert and will answer a coding-related question.
Analyze the question carefully and provide a **concise yet insightful** response.

### Response Format:
- **Step 1:** Identify key aspects of the problem.
- **Step 2:** Explain your approach in a **clear and structured** way (max 3 sentences).
- **Step 3:** If applicable, mention any edge cases or optimizations.

### Question:
{}

### Response:
"""

confirm_template = """You are a programming expert and will answer multiple-choice questions about code.  
Read the following question and how to solve carefully and select the **best** answer.

### Response Format:
- Reply with **only** the letter of the correct choice (A, B, C, or D).  
- Do **not** provide explanations. 

### Question:
{}

### How to solve:
{}

### Choices:
{}

### Response:
"""

In [69]:
index2choice = {i: letter for i, letter in enumerate(string.ascii_uppercase)}
choice2index = {letter: i for i, letter in enumerate(string.ascii_uppercase)}

def choices2str(choices):
    choices_lst = ast.literal_eval(choices)
    result = ""
    for i in range(0, len(choices_lst)):
        result = result + index2choice[i] + ". " + str(choices_lst[i]) + "\n\n"
    return result.strip()

def apply_solve_template(question):
    return solve_template.format(question)

def apply_confirm(question, how_to_solve, choices):
    return confirm_template.format(question, how_to_solve, choices2str(choices))

In [70]:
print(apply_solve_template(df_train['question'].iloc[0]))

You are a programming expert and will answer a coding-related question.
Analyze the question carefully and provide a **concise yet insightful** response.

### Response Format:
- **Step 1:** Identify key aspects of the problem.
- **Step 2:** Explain your approach in a **clear and structured** way (max 3 sentences).
- **Step 3:** If applicable, mention any edge cases or optimizations.

### Question:
Question: What will be output of the following code?
#include<stdio.h>
int main()
{
    printf("%d\t",sizeof(6.5));
    printf("%d\t",sizeof(90000));
    printf("%d",sizeof('A'));
    return 0;
}
['8 4 2', '8 4 2', '8 4 4', '8 4 3']

Answer: 

### Response:



In [71]:
df_train['template'] = df_train.apply(lambda x: apply_solve_template(x['question']), axis=1)
df_train.head()

,task_id,question,choices,answer,template
0,k10168,Question: What will be output of the following...,"['8 4 2', '8 4 2', '8 4 4', '8 4 3']",C,You are a programming expert and will answer a...
1,k10173,Question: What will be output of the following...,"['-4', '-5', '10', '11']",A,You are a programming expert and will answer a...
2,k10174,Question: Match the following.\n Group 1 ...,"['P-4. Q-1, R-2, S-3', 'P-3, Q-1, R-4, S-2', '...",B,You are a programming expert and will answer a...
3,k10175,Question: Match the following.\nP. Regular exp...,"['P-4. Q-1, R-2, S-3', 'P-3, Q-1, R-4, S-2', '...",B,You are a programming expert and will answer a...
4,k10176,Question: Which grammar rules violate the requ...,"['1 only', '1 and 3 only', '2 and 3 only', '3 ...",B,You are a programming expert and will answer a...


In [72]:
df_test['template'] = df_test.apply(lambda x: apply_solve_template(x['question']), axis=1)
df_test.head()

,task_id,question,choices,template
0,k09698,Question: Consider the following array of elem...,"['4', '5', '2', '3']",You are a programming expert and will answer a...
1,k00203,Question: The SVG <polygon> element is used to...,"['False', 'True']",You are a programming expert and will answer a...
2,k00137,Question: Only one of the following statements...,['The view SHOULD NOT be affected by the chang...,You are a programming expert and will answer a...
3,k10490,Question: Which operator is used to multiply n...,"['x', '%', '*', '#']",You are a programming expert and will answer a...
4,rt03960,"Question: Given a code snippet below, which be...","['Compile Error', 'Runtime Error', 'Internal e...",You are a programming expert and will answer a...


# Response Parser

In [73]:
def response_parser(response):
    if len(response) == 1 and response.upper() in list(choice2index.keys()):
        return response

    response = response.translate(str.maketrans('', '', string.punctuation))
    response_split = response.split()
    for re in response_split:
        if re in list(choice2index.keys()):
            return re
    return random.choice(list(choice2index.keys()))

# Inference

In [74]:
def generate(prompt:str, max_new_tokens=1):
        inputs = tokenizer(
            [prompt], return_tensors="pt"
        ).to("cuda")
        input_length = inputs.input_ids.shape[1]  # Lưu độ dài ban đầu
        
        output_ids = model.generate(**inputs, max_new_tokens=max_new_tokens, use_cache=True)
        new_tokens = output_ids[0, input_length:]
        response = tokenizer.decode(new_tokens, skip_special_tokens=True)

        return response.strip()

In [79]:
data_dict = {
    "task_id": [],
    "answer": []
}

In [ ]:
for i in tqdm(range(0, df_test.shape[0]), desc="Inference on Test set"):
    task_id = df_test['task_id'].iloc[i]
    prompt = df_test['template'].iloc[i]
    how_to_solve = generate(prompt, 400)
    final_prompt = apply_confirm(df_test['question'].iloc[i], how_to_solve, df_test['choices'].iloc[i])
    response = generate(final_prompt, 2)
    data_dict['task_id'].append(task_id)
    answer = response_parser(response)
    data_dict['answer'].append(answer)

Inference on Test set:   0%|          | 0/1253 [00:00<?, ?it/s]

# Submission

In [ ]:
submission = pd.DataFrame(data_dict)
submission.head()

In [ ]:
submission.describe()

In [ ]:
submission.to_csv("submission.csv", index=False)